# MarketIndicators.jl

*by [Uki D. Lucas](https://www.linkedin.com/in/ukidlucas/)*

This project was started in August 2020 on [GitHub](https://github.com/UkiDLucas/MarketIndicators.jl)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#MarketIndicators.jl" data-toc-modified-id="MarketIndicators.jl-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>MarketIndicators.jl</a></span></li><li><span><a href="#Overview" data-toc-modified-id="Overview-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Overview</a></span><ul class="toc-item"><li><span><a href="#What-are-Market-Indicators?" data-toc-modified-id="What-are-Market-Indicators?-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>What are Market Indicators?</a></span></li><li><span><a href="#Project-Objectives" data-toc-modified-id="Project-Objectives-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Project Objectives</a></span></li><li><span><a href="#Why-using-programming,-not-spreadsheets?" data-toc-modified-id="Why-using-programming,-not-spreadsheets?-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Why using programming, not spreadsheets?</a></span><ul class="toc-item"><li><span><a href="#AI" data-toc-modified-id="AI-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>AI</a></span></li></ul></li><li><span><a href="#Why-Julia?" data-toc-modified-id="Why-Julia?-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Why Julia?</a></span></li></ul></li><li><span><a href="#Data-Aquisition" data-toc-modified-id="Data-Aquisition-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Aquisition</a></span><ul class="toc-item"><li><span><a href="#Learn-about-market-indicators" data-toc-modified-id="Learn-about-market-indicators-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Learn about market indicators</a></span></li><li><span><a href="#Download-from-Yahoo-Finance" data-toc-modified-id="Download-from-Yahoo-Finance-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Download from Yahoo Finance</a></span></li><li><span><a href="#Download-from-mql5.com" data-toc-modified-id="Download-from-mql5.com-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Download from mql5.com</a></span></li></ul></li><li><span><a href="#Data-Pre-processing" data-toc-modified-id="Data-Pre-processing-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data Pre-processing</a></span><ul class="toc-item"><li><span><a href="#Common-date-formatting" data-toc-modified-id="Common-date-formatting-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Common date formatting</a></span><ul class="toc-item"><li><span><a href="#Explanation-of-terms" data-toc-modified-id="Explanation-of-terms-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>Explanation of terms</a></span></li><li><span><a href="#yy-mm-dd" data-toc-modified-id="yy-mm-dd-4.1.2"><span class="toc-item-num">4.1.2&nbsp;&nbsp;</span>yy-mm-dd</a></span></li><li><span><a href="#yy.mm.dd" data-toc-modified-id="yy.mm.dd-4.1.3"><span class="toc-item-num">4.1.3&nbsp;&nbsp;</span>yy.mm.dd</a></span></li><li><span><a href="#u-dd,-yyyy" data-toc-modified-id="u-dd,-yyyy-4.1.4"><span class="toc-item-num">4.1.4&nbsp;&nbsp;</span>u dd, yyyy</a></span></li></ul></li><li><span><a href="#Rata-Die-(days-since)" data-toc-modified-id="Rata-Die-(days-since)-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Rata Die (days since)</a></span></li><li><span><a href="#Populate-missing-indicators-with-previously-known-values" data-toc-modified-id="Populate-missing-indicators-with-previously-known-values-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Populate missing indicators with previously known values</a></span></li></ul></li><li><span><a href="#Normalize-data" data-toc-modified-id="Normalize-data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Normalize data</a></span></li></ul></div>

# Overview

## What are Market Indicators?

Market Indicators are numeric values assigned to particular aspect of the market performance, for example:
- "S&P 500" index show how major stocks as a whole are performing
- "ISM Manufacturing" index shows how well the manufacturing industry is doing
- "GDP" index (Gross Domestic Product) shows how the country is doing
- etc., etc., etc.

There are hundreds, if not thousands, of such indicators.

## Project Objectives

1. The primary goal of the project is to **gather and analyze** various market indicators.
2. The secondary goal is to predict market trends 30 to 90 days ahead. 
3. The tertiary goal is to predict a particular stock price based on the market indicators.
4. Additionally, the goal will be to maintain a stock portfolio including diversification using "hedge fund" principles.
5. Finally, it is my goal to document the process.

## Why using programming, not spreadsheets?

The spredsheets (i.e. Microsoft Excel, Apple Numbers, Google Sheets) and their graphing capability are the bread and butter of market analysis. They are great when comparing a few indicators.

### AI
The advantage of machine learning, or Artificial Intelligence (AI), is when that we have thousands of possible indicators, the human brain fails to grasp the wealth of the information presented. 

The machine learning, on the other hand can easily detect the patterns in thousands of datapoints and derive a conclussion.

## Why Julia?

- Julia is young and sexy (we are talking about [Julia programming language](https://docs.julialang.org/en/v1/) here)
- it is a pleasure to write and read (unlike C,C++)
- it is designed for scientific computing and machine learning
- it is extremely fast (similarly to C)
- it is very easy (similary to Python)
- it is designed for parallelism
- it is designed for distributed computing

# Data Aquisition

## Learn about market indicators
- [indicators that move the market](https://www.investopedia.com/articles/fundamental-analysis/10/indicators-that-move-the-market.asp)

## Download from Yahoo Finance

- [Yahoo Finance > Historical Data > Max](https://finance.yahoo.com/quote/%5EDJI/history?p=%5EDJI)
- [Apple AAPL - finance.yahoo.com]("https://finance.yahoo.com/quote/AAPL/history?p=AAPL")

## Download from mql5.com

-[mql5.com](https://www.mql5.com/en/economic-calendar/united-states)


# Data Pre-processing


## Common date formatting

The dates in the spreadsheets, especially these that are entered by hand, have a maddening array of possible formats.

The task of getting all the dates to the same format is crucial and it is done fastest with the spreadsheet.

### Explanation of terms
- u: 3 letter abbreviation of the month in English
- d: 1 or 2 digit day of the month
- y: 4 digit year
- m: 2 digit month (option for you)

### yy-mm-dd
The date format **I chose** is "1950-05-31", or (yy-mm-dd) as it is **less error prone, logical, and easily sortable**.
This format is also used by Yahoo Finance export.

$Dates.Date("1950-05-31", date_format) # results in "1950-05-31"$

### yy.mm.dd
This format is used by MQL5.com, a primary source of indicator data.


### u dd, yyyy
As an example, the standard in USA is "Aug 31, 1950" (u d, y) 



## Rata Die (days since)

Using dates is useful, but the market indicators are not availble on daily basis, most come out bi-weekly, mothly or quarterly, we will have some days that are "missing" in our datasets.

We will assign "[Rata Die](https://en.wikipedia.org/wiki/Rata_Die)", or the "days since the first day" in our data. 

I am using the first day of available data and not some other major date, such as founding of the New York Stock Exchange in 1800s as that numbers would be huge and there would be no useful early data anyway.

## Populate missing indicators with previously known values

- get value for the first row
- for each row starting at row 2
    - get the current value
        - if that value is missing
            - set it to the previus value if you have it
        - if you have current row value
            - set the previous to be equal to current
    
actual code:

# Normalize data

Let's say we have two market indicators, one ranges from -1 to 1 and another from 0 to 100. Obviously it is not easy to compare them to see how they influence each other. 

![title](images/not_normalized.png)


Normalization is nothing else than "scaling" the data to be able to compare apples-to-apples.

![title](images/normalized.png)